# IMPORT & INITIALISATIONS

In [44]:
# !pip install spacy
# !python -m spacy download en_core_web_sm
# !python -m spacy download en
# !pip install plotly==5.6.0
# !pip install dash
# !pip install "jupyterlab>=3" "ipywidgets>=7.6"
# !pip install chart-studio==1.1.0
# !pip install psutil

In [1]:
import spacy
import re
import collections
import pandas as pd
import numpy as np
import plotly.express as px 
from collections import Counter

In [2]:
def clean_text(text):
    temp = text.lower()
    temp = re.sub("'", "", temp) # to avoid removing contractions in english
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    temp = re.sub("[^a-z0-9]"," ", temp)
#     temp = temp.split()
#     temp = [w for w in temp if not w in stopwords]
#     temp = " ".join(word for word in temp)
    return temp

In [3]:
# Function to get unique words for each aspect 
def unique_words(aspect_dict, aspect): 
    unique_words_dict = {}
    unique_word_list = []
    for desc in aspect_dict[aspect]:
        if desc not in unique_word_list:
            unique_word_list.append(desc)
        
    return unique_word_list

#get the count for each unique word for each aspect 
def get_counts_desc_list(aspect_dict, aspect):
    count_desc = {}
    count_list = []

    
    for desc in aspect_dict[aspect]:
        if desc not in count_desc:
            count_desc[desc] = 1
        else:
            count_desc[desc] += 1 
    
    for desc in count_desc:
        count_list.append(count_desc[desc])

                        
    
    return count_list 


def get_proportion_desc_list(aspect_dict,aspect):
    count_desc = {}
    count_list = []
    proportion_list = []
    total= 0 
    
    
    for desc in aspect_dict[aspect]:
        if desc not in count_desc:
            count_desc[desc] = 1
        else:
            count_desc[desc] += 1 
    
    for desc in count_desc:
        total += count_desc[desc]
    for desc in count_desc:
        proportion_list.append(count_desc[desc]/total)

                        
    
    return proportion_list 
        

In [4]:
nlp = spacy.load("en_core_web_sm")

# GETTING THE DATA

###### MENTAIYA: 
pos_reviews1 = pd.read_excel(r"C:\Users\kytan\OneDrive\Desktop\SMUACADS\Y2S2\SA\Projects\github\IS434_G1T5\data\Competitorsdata_ABSA\mentaiya_cleaned_pos_reviews.xlsx", names=['Reviews'])
neg_reviews1 = pd.read_excel(r"C:\Users\kytan\OneDrive\Desktop\SMUACADS\Y2S2\SA\Projects\github\IS434_G1T5\data\Competitorsdata_ABSA\mentaiya_cleaned_neg_reviews.xlsx", names=["Reviews"])

###### FUKUDON

pos_reviews1= pd.read_excel(r"C:\Users\kytan\OneDrive\Desktop\SMUACADS\Y2S2\SA\Projects\github\IS434_G1T5\data\Competitorsdata_ABSA\fukudon_cleaned_pos_reviews.xlsx", names=['Reviews'])

neg_reviews1= pd.read_excel(r"C:\Users\kytan\OneDrive\Desktop\SMUACADS\Y2S2\SA\Projects\github\IS434_G1T5\data\Competitorsdata_ABSA\fukudon_cleaned_neg_reviews.xlsx", names=['Reviews'])


###### STICKSNBOWLS 
pos_reviews1 = pd.read_csv(r"C:\Users\kytan\OneDrive\Desktop\SMUACADS\Y2S2\SA\Projects\github\IS434_G1T5\data\Competitorsdata_ABSA\stickNbowls_cleaned_pos_reviews.csv", names=['Reviews'])
neg_reviews1 = pd.read_csv(r"C:\Users\kytan\OneDrive\Desktop\SMUACADS\Y2S2\SA\Projects\github\IS434_G1T5\data\Competitorsdata_ABSA\stickNbowls_cleaned_neg_reviews.csv", names=["Reviews"]) -->

In [2]:
# MENTAIYA
pos_reviews1 = pd.read_excel(r"C:\Users\kytan\OneDrive\Desktop\SMUACADS\Y2S2\SA\Projects\Competitorsdata\mentaiya_cleaned_pos_reviews.xlsx", names=['Reviews'])
neg_reviews1 = pd.read_excel(r"C:\Users\kytan\OneDrive\Desktop\SMUACADS\Y2S2\SA\Projects\Competitorsdata\mentaiya_cleaned_neg_reviews.xlsx", names=["Reviews"])

In [65]:
# STICKSNBOWLS 
pos_reviews1 = pd.read_csv(r"C:\Users\kytan\OneDrive\Desktop\SMUACADS\Y2S2\SA\Projects\Competitorsdata\stickNbowls_cleaned_pos_reviews.csv", names=['Reviews'])
neg_reviews1 = pd.read_csv(r"C:\Users\kytan\OneDrive\Desktop\SMUACADS\Y2S2\SA\Projects\Competitorsdata\stickNbowls_cleaned_neg_reviews.csv", names=["Reviews"])

In [5]:
# FUKUDON
pos_reviews1= pd.read_excel(r"C:\Users\kytan\OneDrive\Desktop\SMUACADS\Y2S2\SA\Projects\Competitorsdata\fukudon_cleaned_pos_reviews.xlsx", names=['Reviews'])
neg_reviews1= pd.read_excel(r"C:\Users\kytan\OneDrive\Desktop\SMUACADS\Y2S2\SA\Projects\Competitorsdata\fukudon_cleaned_neg_reviews.xlsx", names=['Reviews'])

In [8]:
# remove repeats for each df in case have
pos_reviews = pos_reviews1.drop_duplicates("Reviews", inplace = False)
pos_reviews.reset_index(inplace=True)
neg_reviews = neg_reviews1.drop_duplicates("Reviews")
neg_reviews.reset_index(inplace=True)

In [9]:
pos_reviews= pos_reviews.drop(columns="index")
neg_reviews = neg_reviews.drop(columns="index")

# POSITIVE REVIEWS

In [10]:
pos_reviews.tail()

,Reviews
70,ordered company gathering food came hot fresh ...
71,pork n chucky fries nicer n fries highly recom...
72,good food better previous stall
73,food prices affordable
74,run good guys alliance


In [11]:
# PUT INTO A LIST 
pos_reviews_list = pos_reviews["Reviews"].to_list()

In [12]:
pos_aspect_list = []

for sentence in pos_reviews_list:
    s = clean_text(sentence)
    doc = nlp(s)
    descriptive_term = ''
    aspects = []
    for token in doc:
        if token.pos_ == 'ADJ' :
            descriptive_term = token
        if token.pos_ == "NOUN" or token.pos_ == "PROPN":
            aspects.append(str(token))
    pos_aspect_list += aspects
    
            
#     print("===========================================")
#     print("************** Aspects **************")
#     print(", ".join(aspects)) #get the noun of sentence
#     print("************** Description ***************")
#     print(descriptive_term)
#     print()

In [13]:
# COUNTING ITEMS 
pos_freq = collections.Counter(pos_aspect_list)
print(pos_freq)
# # print("#######################################################################################################################")
# top_pos= dict(collections.Counter(pos_freq))

Counter({'food': 36, 'pork': 16, 'sauce': 11, 'price': 10, 'belly': 9, 'katsu': 9, 'portion': 9, 'hawker': 9, 'curry': 8, 'rice': 7, 'love': 7, 'tender': 7, 'tasty': 7, 'beef': 7, 'fries': 7, 'egg': 6, 'time': 6, 'chicken': 6, 'stall': 6, 'donburi': 6, 'delivery': 5, 'karaage': 5, 't': 5, 'taste': 5, 'salad': 4, 'spicy': 4, 'collar': 4, 'crispy': 4, 'order': 4, 'bowl': 4, 'service': 3, 'punggol': 3, 'platter': 3, 'onsen': 3, 'try': 3, 'dish': 3, 'guys': 3, 'eggs': 3, 'onion': 3, 'texture': 3, 'thunder': 3, 'crunchy': 3, 'place': 3, 'prices': 3, 'delicious': 3, 'area': 3, 'queue': 3, 'fukudon': 3, 'quality': 3, 'salmon': 3, 'restaurant': 3, 'portions': 2, 'good': 2, 'jap': 2, 'ingredients': 2, 'bite': 2, 'mention': 2, 'gem': 2, 'marine': 2, 'parade': 2, 'meat': 2, 'chunks': 2, 'tomatoes': 2, 'cucumber': 2, 'goma': 2, 'components': 2, 'flavour': 2, 'mentaiko': 2, 'center': 2, 'garlic': 2, 'w': 2, 'homemade': 2, 'amount': 2, 'm': 2, 'support': 2, 'sukiyaki': 2, 'store': 2, 'plate': 2, 'to

In [6]:
top_pos_aspects = []
top_pos =dict(pos_freq)
for key in top_pos:
    if top_pos[key] >= (len(pos_reviews)*0.10):
        print(key)
        top_pos_aspects.append(key)

In [25]:
print(top_pos_aspects)

['food', 'delivery', 'pork', 'belly', 'katsu', 'rice', 'egg', 'portion', 'time', 'hawker', 'love', 'chicken', 'karaage', 'sauce', 't', 'tender', 'tasty', 'salad', 'stall', 'beef', 'fries', 'curry', 'spicy', 'donburi', 'collar', 'price', 'crispy', 'taste', 'order', 'bowl']


### COMPILING INTO DF - POSITIVE 

#### Two types of DF created: 
1. Sentence --> Aspect (purpose is to see which sentence pointed to the aspects)
2. Aspect --> Descriptive words (Purpose is to see what each aspect was described)

We chose to use the second one for our analysis. 

### 1.Create a dictionary that points sentence to aspect then make into a df

In [26]:
# creating a sentence-aspect dictionary 
pos_sent_aspect_dict = {}
for sentence in pos_reviews_list:
    s = clean_text(sentence)
    doc = nlp(s)
    descriptive_term = ''
    aspects = []
    for token in doc:
        if token.pos_ == 'ADJ' :
            descriptive_term = token
        if token.pos_ == "NOUN" or token.pos_ == "PROPN" or token.pos_ =="VERB":
            aspects.append(str(token)) #for each sentence, store all the aspects in a list 
            if str(token) in top_pos_aspects:
                pos_sent_aspect_dict[s] = str(token)    

In [27]:
pos_df = pd.DataFrame.from_dict(pos_sent_aspect_dict, orient='index', columns=['Aspect'])
pos_df.to_csv("pos_sent_asp_fukudon.csv")

### 2. Create a dict then df for aspect-descriptive

In [28]:
# Aim to create a asepct-descriptive term df 

# initialisation
temp_pos_asp_descrp_dict = {}
asp = ''
# code 
for sentence in pos_reviews_list:
    s = clean_text(sentence)
    doc = nlp(s)
    descriptive_term = ''

    for token in doc:
        if token.pos_ == 'ADJ' :
            descriptive_term = str(token)
        if token.pos_ == "NOUN" or token.pos_ == "PROPN":
            asp = str(token)
        
        if asp in top_pos_aspects:
            if descriptive_term != '':
                if asp not in temp_pos_asp_descrp_dict:
                    temp_pos_asp_descrp_dict[asp] = descriptive_term
                else:
                    temp_pos_asp_descrp_dict[asp] += ", " + descriptive_term


In [29]:
pos_aspect_descrp_df = pd.DataFrame.from_dict(temp_pos_asp_descrp_dict, orient='index')

In [30]:
# uncomment if you want to visualise to see output 
print(pos_aspect_descrp_df)

                                                          0
food      delicious, prompt, japanese, less, awesome, aw...
delivery  prompt, excellent, awesome, free, minimum, fre...
pork      friendlyordered, expected, impressed, complica...
belly           friendlyordered, expected, impressed, super
katsu     friendlyordered, friendlyordered, delicious, i...
rice      extra, good, plentiful, cheap, cheap, cheap, c...
egg       extra, less, less, complicating, short, genero...
portion   hot, hot, generous, worth, good, double, good,...
time      affordable, best, soggy, soggy, expected, next...
hawker    best, central, younger, particular, humble, fa...
love                             good, healthy, donburimade
chicken      good, complicating, light, double, good, inner
karaage        good, complicating, pricey, sexy, sexy, sexy
sauce     good, least, least, least, plentiful, japanese...
tender    expected, marinated, unctuous, interested, lig...
tasty     expected, soft, scrambled, aff

### POSITIVE VISUALISATION
Use plotly:https://plotly.com/python/getting-started/ 

For each aspect, get unique description words (x-axis) and the frequency (y-axis).

In [31]:
# Put description as a list for each aspect in top_5_neg_aspects:
pos_asp_dict = {}
for asp in temp_pos_asp_descrp_dict:
    content = temp_pos_asp_descrp_dict[asp]
    to_list = content.split(',')
    pos_asp_dict[asp] = to_list

# #combine 2 similar words
# pos_asp_dict['mentaiko'] += pos_asp_dict['mentai']
# del pos_asp_dict['mentai'] #delete the one added 

In [3]:
# BAR CHART 
for asp in pos_asp_dict:
    x_list = []
    y_list = []
    for x,y in zip(unique_words(pos_asp_dict,asp),get_counts_desc_list(pos_asp_dict, asp)):
        if y>1:
            x_list.append(x)
            y_list.append(y)
    if len(x_list)!=0:
        fig = px.bar(x= x_list, y= y_list, title="Positive: " + asp, labels=dict(x ="Description",y= "Frequency"))
        fig.update_layout(xaxis={'categoryorder':'total descending'})
        fig.show()

In [5]:
# PIE CHART
for asp in pos_asp_dict:
    df = pd.DataFrame(unique_words(pos_asp_dict,asp), columns=['Description'])
    df['counts'] = get_counts_desc_list(pos_asp_dict, asp)
    fig = px.pie(df, values= 'counts', names='Description', title='Positive: '+ asp)
    fig.show()

# NEGATIVE REVIEWS

In [241]:
neg_reviews_list = neg_reviews["Reviews"].to_list()  #convert to list 

In [242]:
neg_aspect_list = []
neg_appended = []
# sentence_list = []
# a_list = []
for sentence in neg_reviews_list:
    s = clean_text(sentence)
    doc = nlp(s)
    descriptive_term = ''
    aspects = []
    for token in doc:
        if token.pos_ == 'ADJ' :
            descriptive_term = token
        if token.pos_ == "NOUN" or token.pos_ == "PROPN":
            aspects.append(str(token))
    neg_appended.append(aspects)        

    neg_aspect_list += aspects
    

# UNCOMMENT THE FOLLOWING TO SEE ASPECTS AND DESCRIPTION OUTPUTS
            
#     print("===========================================")
#     print("************** Aspects **************")
#     print(", ".join(aspects)) #get the noun of sentence
#     print("************** Description ***************")
#     print(descriptive_term)
#     print()

##### FOR COUNTING OF ITEMS:
Set line 5 to 0.10 to make sure that the aspects gathered appear at least 10% of all the reviews

In [245]:
# COUNTING ITEMS 
neg_freq = collections.Counter(neg_aspect_list)
top_neg_aspects = []
top_neg_dict =dict(neg_freq)
for key in top_neg_dict:
    if top_neg_dict[key] >= (len(neg_reviews)*0.10):
        top_neg_aspects.append(key)

### COMPILING INTO DF - NEGATIVE

#### Two types of DF created: 
1. Sentence --> Aspect (purpose is to see which sentence pointed to the aspects)
2. Aspect --> Descriptive words (Purpose is to see what each aspect was described)

We chose to use the second one for our analysis. 

### 1.Create a dictionary that points sentence to aspect then make into a df

In [247]:
neg_sent_aspect_dict = {}
for sentence in neg_reviews_list:
    s = clean_text(sentence)
    doc = nlp(s)
    descriptive_term = ''
    aspects = []
    for token in doc:
        if token.pos_ == 'ADJ' :
            descriptive_term = token
        if token.pos_ == "NOUN" or token.pos_ == "PROPN":
            aspects.append(str(token)) #for each sentence, store all the aspects in a list 
            if str(token) in top_neg_aspects:
#                 if str(token) not in sent_aspect_dict:
                neg_sent_aspect_dict[s] = str(token)
#                 else:
#                     sent_aspect_dict[str(token)] += s 
#     neg_appended.append(aspects)        

In [248]:
neg_df = pd.DataFrame.from_dict(neg_sent_aspect_dict, orient='index', columns=['Aspect'])

In [249]:
neg_df

,Aspect
super valued money super affordable yet restaurant standard sukiyaki beef don returning weekly satisfy cravings yes friendly staffs too,beef
place certainly worthy tried pork katsu beef fell way expectations katsu soggy beef toughhad marine parade outletpunggol outlet might better,outlet
picture doesn t food justice fav place pork spicy super fragrant super appetising would crave sometimes tried dons including salmon chicken oyakodon katsu really rather worth price point salmon offered tomato salad good katsu might get abit soggy longer crispy would reco going takeaway,takeaway
marine parade saw stall selling japanese constant queue decided try person took order wrote slip paper order order number gave one slip slip person preparing food cashier would shout order number order done would advise ordering get seats near stall know food readyi got teriyaki beef short plate w onsen egg consists beef short plate onions onsen egg furikakeso think dish come onions either minced onions really fine rice rice bad drenched beef drippings added flavour plain rice beef tender coated teriyaki sauce topped furikake furikake think add much value entire dish mostly tasted togarashi japanese chili powder onsen egg little disappointed yolk already brokeoverall average tasting bowl beef felt could given slightly meat maybe add furikake rice instead meat least people eat rice taste,taste
folks fukudonsg got mise en place kitchen line tee enabling serve steady stream customers quickly doesn t always work favour beef tender unctuous onsen egg yolk already overcooked nevertheless spot flavours although always rice bowls could definitely saucetaste,bowls
id hardpressed find another place serving salmon quality price point budget friendly yet tasty donburis queue starts forming way shutters go lunch everyday nooneach serving comes pieces succulent oily salmon coated tangy shoyu lemon glaze side fukudonsg japanese tomato salad bed al dente calrose rice portion small every bite flavour packed want get hands come it s one bestsellers run inevitablytaste,flavour
asn t intending settle dinner marine parade food centre initially pretty bummed fact spot intended dine area many items menu unavailable thus found heading food centre checking fukudon describes asian fusion donburi specialist following social media quite whilefelt asian fusion donburi branding bit far stretch given various donburi offered menu ie chicken breast oyakodon shoyu glaze salmon sukiyaki beef short plate onsen eg etc sound pretty japaneseinspired said impressed pork belly katsu opted fried pork belly seems carry bite less gelatinous fatty initially expected especially given illustrations menu element sealed deal us runny egg simmered dashi broth alongside onions incredibly runny silky eggs also carried punchy savoury note provides much flavour entire dish sauce even permeating bed shortgrain rice gives otherwise plain rice moisture sweetsavoury note japanese spicy powder also sprinkled top provide donburi slight hint spiciness enough tickle tastebuds without overwhelming entire dishgiven tried fukudon left pretty impressed offer execution eggs particularly noteworthy pretty much important element makes good oyakodon katsudon it s offerings seem rather japanese heart seem places quite bit effort trying serve quality food masses pocketfriendly prices quite interested thunder crunchy fries comes either homemade curry sauce mentaiko sauce something pretty keen try subsequent visit,visit
saw new stall marine parade food centre operating bunch young people serving donburi food courtalthough opened recently attracted long queue peak hoursgo signature pork belly katsu topped slices thick crispy juicy pork katsu simmered together egg onion japanese ricethe portion wasn t big sufficient,t
howeverii wish least sauce go along plentiful rice,rice
onsen egg yolk already overcooked,yolk


### 2. Create a dict then df for aspect-descriptive

In [250]:
# Aim to create a asepct-descriptive term df 

# initialisation
temp_neg_asp_descrp_dict = {}

# code 
for sentence in neg_reviews_list:
    s = clean_text(sentence)
    doc = nlp(s)
    descriptive_term = ''
    aspects = []
    
    for token in doc:
        if token.pos_ == 'ADJ' :
            descriptive_term = str(token)
        if token.pos_ == "NOUN" or token.pos_ == "PROPN" or token.pos_ == "VERB":
            asp = str(token)
            aspects.append(str(token)) #for each sentence, store all the aspects in a list 
            
        if asp in top_neg_aspects:
            
            if descriptive_term != '':
                
                if asp not in temp_neg_asp_descrp_dict:
                    temp_neg_asp_descrp_dict[asp] = descriptive_term
                    
                else:
                    temp_neg_asp_descrp_dict[asp] += ',' + descriptive_term


In [251]:
neg_asp_descrp_df = pd.DataFrame.from_dict(temp_neg_asp_descrp_dict, orient='index',columns=['Description'])
neg_asp_descrp_df

,Description
money,"affordable,affordable,great,great,great"
sukiyaki,"standard,shoyu,average,central"
beef,"standard,worthy,worthy,constant,short,short,sh..."
place,"worthy,huge,soft"
pork,"worthy,impressed,impressed,long,thick,fried,di..."
...,...
amount,"large,generous"
texture,"average,average,socalled"
time,first
christmas,"central,central"


### NEGATIVE VISUALISATION


Use plotly ==> https://plotly.com/python/getting-started/ 
For each aspect, get unique description words (x-axis) and the count(y-axis).

In [252]:
# Put description as a list for each aspect in top_5_neg_aspects:
neg_asp_dict = {}
for asp in temp_neg_asp_descrp_dict:
    content = temp_neg_asp_descrp_dict[asp]
    to_list = content.split(',')
    neg_asp_dict[asp] = to_list

# #combine 2 similar words HERE
# neg_asp_dict['mentaiko'] += neg_asp_dict['mentai']
# del neg_asp_dict['mentai']

#### Bar Chart

In [8]:
for asp in neg_asp_dict:
    x_list = []
    y_list = []
    for x,y in zip(unique_words(neg_asp_dict,asp),get_counts_desc_list(neg_asp_dict, asp)):
        if y>1:
            x_list.append(x)
            y_list.append(y)
    if len(x_list)!=0:
        fig = px.bar(x= x_list, y= y_list, title="Negative: " + asp, labels=dict(x ="Description",y= "Frequency"))
        fig.update_layout(xaxis={'categoryorder':'total descending'})
        fig.show()

#### PIe Chart

In [7]:
for asp in neg_asp_dict:
    df = pd.DataFrame(unique_words(neg_asp_dict,asp), columns=['Description'])
    df['counts'] = get_counts_desc_list(neg_asp_dict, asp)
    fig = px.pie(df, values= 'counts', names='Description', title='Negative: '+ asp)
    fig.show()